In [3]:
import os
import time
#import modin.pandas as pd





In [4]:
os.path.getsize('E:\Anitha\Data_Glacier\GitHub\VC\Week6\Rate.csv')

13899

In [5]:
#Read in the data with Dask
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('E:\Anitha\Data_Glacier\GitHub\VC\Week6\Rate.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.16183996200561523 sec


In [6]:
#Read in the data with Pandas
import pandas as pd
start = time.time()
df = pd.read_csv('E:\Anitha\Data_Glacier\GitHub\VC\Week6\Rate.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  0.016956806182861328 sec


In [7]:
pip install modin

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install ray

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [9]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('E:\Anitha\Data_Glacier\GitHub\VC\Week6\Rate.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2022-11-09 07:08:27,291	INFO worker.py:1518 -- Started a local Ray instance.


Read csv with modin and ray:  1.5668084621429443 sec


In [10]:
#Here Dask is better than Pandas, Modin and Ray

from dask import dataframe as dd
df = dd.read_csv('E:\Anitha\Data_Glacier\GitHub\VC\Week6\Rate.csv',delimiter=',')


In [11]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, BusinessYear to RowNumber
dtypes: object(9), float64(9), int64(6)

In [12]:
len(df.index)


67

In [19]:
#No, of Columns
len(df.columns)

24

In [18]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [19]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [20]:
data=df.columns
data

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')

In [31]:
 pip install pyyaml

Note: you may need to restart the kernel to use updated packages.


In [32]:
 pip install configparser

Note: you may need to restart the kernel to use updated packages.


In [24]:
# Validation
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import configparser



In [70]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Overwriting utility.py


In [71]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - BusinessYear
      - StateCode
      - IssuerId
      - SourceName
      - VersionNum
      - ImportDate
      - IssuerId2
      - FederalTIN
      - RateEffectiveDate
      - RateExpirationDate
      - PlanId
      - RatingAreaId
      - Tobacco
      - Age
      - IndividualRate
      - IndividualTobaccoRate
      - Couple
      - PrimarySubscriberAndOneDependent
      - PrimarySubscriberAndTwoDependents
      - PrimarySubscriberAndThreeOrMoreDependents
      - CoupleAndOneDependent
      - CoupleAndTwoDependents
      - CoupleAndThreeOrMoreDependents
      - RowNumber

Overwriting store.yaml


In [53]:
import datetime
import csv
import gzip

from dask import dataframe as ddf
df1 = dd.read_csv('E:\Anitha\Data_Glacier\GitHub\VC\Week6\Rate.csv')

# Write csv in gz format in pipe separated text file (|)
df1.to_csv('Rate.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['C:/Users/sazha/Rate.csv.gz\\0.part']

In [58]:
#number of files in gz format folder
import os
files = os.listdir(r'C:\Users\sazha\Rate.csv.gz')
for file in files:
    print(file)

0.part


In [10]:
#size of the gz format folder
#os.path.getsize(r'C:\Users\sazha\Rate.csv.gz')
size = os.path.getsize(r'C:\Users\sazha\Rate.csv.gz')
print(size)
                
                
                
                
                
                
                
                

0
